In [1]:
#Import data

In [42]:
import numpy as np
import pandas as pd
# Read file with param './data/train.csv', encoding='latin-1'

path = './Downloads/Data/ny.csv'
path2 ='./Downloads/Data/ny3.csv'
path3 ='./Downloads/Data/nyc2014.csv'
path4 ='./Downloads/Data/nyc2010.csv'
path5 ='./Downloads/Data/nyc2018.csv'

ny2019 = pd.read_csv(path)
ny2019=ny2019[['status','status_date','description','committee']]
ny2019=ny2019[pd.notnull(ny2019['committee'])]
ny2019=ny2019[pd.notnull(ny2019['description'])]
filtered1= ny2019.loc[ny2019['status'] == 1]


ny2016 = pd.read_csv(path2)
ny2016=ny2016[['status','status_date','description','committee']]
ny2016=ny2016[pd.notnull(ny2016['committee'])]
ny2016=ny2016[pd.notnull(ny2016['description'])]
filtered2= ny2016.loc[ny2016['status'] == 1]

ny2014 = pd.read_csv(path3)
ny2014=ny2014[['status','status_date','description','committee']]
ny2014=ny2014[pd.notnull(ny2014['committee'])]
ny2014=ny2014[pd.notnull(ny2014['description'])]
filtered3= ny2014.loc[ny2014['status'] == 1]

ny2010 = pd.read_csv(path4)
ny2010=ny2010[['status','status_date','description','committee']]
ny2010=ny2010[pd.notnull(ny2010['committee'])]
ny2010=ny2010[pd.notnull(ny2010['description'])]
filtered4= ny2010.loc[ny2010['status'] == 1]

ny2018 = pd.read_csv(path5)
ny2018=ny2018[['status','status_date','description','committee']]
ny2018=ny2018[pd.notnull(ny2018['committee'])]
ny2018=ny2018[pd.notnull(ny2018['description'])]
filtered5= ny2018.loc[ny2018['status'] == 1]



data=pd.concat([filtered1,filtered2,filtered3,filtered4,filtered5])
data.dropna()

# Let's check sentiment.head(10) and sample(10)
# Your code here
# data.sample(3)
print(data.sample(100))

       status status_date                                        description  \
11977       1  2013-01-09  Requires the state university trustees to repo...   
503         1  2013-01-09  Provides that on all publicly accessible munic...   
12243       1  2010-04-16  Enacts the "2010 Campaign Finance Reform Act";...   
5885        1     2/20/19  Enacts the "unmarked burial site protection ac...   
4339        1  2017-02-02  Relates to establishing the private employers ...   
18010       1  2018-01-03  Prohibits the outdoor release of 25 or more ba...   
10365       1     1/11/19  Relates to the use of voice recognition featur...   
1603        1  2017-01-12  Provides that the issue of military deployment...   
6158        1  2010-01-06  Requires the department of education to establ...   
1850        1  2009-01-21  To exempt certain class B multiple dwellings a...   
8969        1  2016-02-18  Relates to the authority of nurse practitioner...   
7622        1  2011-01-05  Relates to th

In [43]:
#check null data
data.isnull().sum()

status         0
status_date    0
description    0
committee      0
dtype: int64

In [45]:
#drop the committee that has low recall, which weighs down accuracy score of the model
df = data[data.committee == 'Investigations and Government Operations']
print(df.index)
df.drop(df.index)

Int64Index([], dtype='int64')


,status,status_date,description,committee


In [47]:
#committee that has invalid description
data['description'][20052]

"Error!: SQLSTATE[42000] [1203] User manage already has more than 'max_user_connections' active connections"

In [48]:
#drop the index with invalid description
data=data.drop([20052])

In [49]:
#check the shape of data
print(data.shape)

(62503, 4)


In [50]:
#check to see if there's a bill without description
print(data[data["description"].isnull()])

Empty DataFrame
Columns: [status, status_date, description, committee]
Index: []


In [52]:
#check the list of all committees
count_val=data.groupby("committee")["committee"].count().sort_values(ascending=False)
count=pd.DataFrame(data=count_val)
print(count.index)

Index(['Assembly Ways and Means', 'Assembly Codes', 'Assembly Transportation',
       'Assembly Education', 'Senate Rules', 'Senate Finance',
       'Assembly Governmental Operations', 'Senate Codes',
       'Senate Investigations and Government Operations', 'Assembly Health',
       'Assembly Judiciary', 'Senate Education', 'Senate Health',
       'Assembly Higher Education', 'Senate Transportation',
       'Assembly Environmental Conservation',
       'Assembly Corporations, Authorities and Commissions',
       'Assembly Insurance', 'Senate Judiciary', 'Senate Local Government',
       'Assembly Real Property Taxation', 'Assembly Labor',
       'Assembly Governmental Employees', 'Assembly Election Law',
       'Assembly Housing', 'Senate Environmental Conservation',
       'Assembly Local Governments',
       'Assembly Economic Development, Job Creation, Commerce and Industry',
       'Senate Consumer Affairs and Protection', 'Senate Insurance',
       'Senate Higher Education',
    

In [53]:
#Clean and unify committees by removing senate, house and assembly
data.replace(to_replace="Assembly Local Governments",value="Senate Local Government", inplace=True)

In [56]:
data=data[['status_date','description','committee']]
data=data[pd.notnull(data['committee'])]
data.drop_duplicates(subset='description',keep='first',inplace=True)
data['committee'] = data['committee'].map(lambda x: x.lstrip('Senate'))
data['committee'] = data['committee'].map(lambda x: x.lstrip('House'))
data['committee'] = data['committee'].map(lambda x: x.lstrip('Assembly'))
data=data[data.groupby("committee")["committee"].transform('size')>400]
data.sample(3)
# print(data.shape)
print(len(data.groupby("committee")["committee"].count()))
print((data.groupby("committee")["committee"].count()).sort_values(ascending=False))

23
committee
 Codes                                                        2462
 Ways and Means                                               2036
 Education                                                    1776
 Transportation                                               1719
 Health                                                       1417
 Governmental Operations                                      1206
 Judiciary                                                    1162
 Rules                                                        1053
 Environmental Conservation                                   1022
 Higher Education                                             1011
 Local Government                                              888
 Corporations, Authorities and Commissions                     826
 Insurance                                                     819
 Labor                                                         746
 Finance                                         

In [15]:
#shape of new data
print(data.shape)

(23217, 3)


In [16]:
#check the number of committees again
labels=list(data['committee'])
myarray=np.asarray(labels)
# print(myarray)
# print(type(myarray))
myarrays=np.unique(myarray)
print(myarrays.shape)

(23,)


In [17]:
#recheck null data
data.isnull().sum()

status_date    0
description    0
committee      0
dtype: int64

In [ ]:
#Clean data and build model

In [20]:
#Step 1: Preprocessing Data

In [21]:
import nltk

nltk.download('stopwords')


[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/tienmai/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [23]:
import re
def preprocessor(text):
    """ Return a cleaned version of text
    """
    # Remove HTML markup
    text = re.sub('<[^>]*>', '', text)
    # Save emoticons for later appending
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text)
    # Remove any non-word character and append the emoticons,
    # removing the nose character for standarization. Convert to lower case
    text = (re.sub('[\W]+', ' ', text.lower()) + ' ' + ' '.join(emoticons).replace('-', ''))
    
    return text

# Create some random texts for testing the function preprocessor()


In [24]:
from nltk.stem import PorterStemmer

porter = PorterStemmer()


# write a function named `tokenizer_porter()` that split a text into list of words and apply stemming technic
# Hint: porter.stem(word)
def tokenizer_porter(text):
    token = []
    # Your code here
    token = [porter.stem(word) for word in text.split()]
    
    return token

In [26]:
data['description'] = data['description'].apply(preprocessor)

In [27]:
#after building the model and checking classification report, I realized there are 3 committees that 
#have significantly low reall score, and therefore weighs down the accuracy of the model. 
#Therefore I decided to remove them
edit=[' Finance',' Investigations and Government Operations',' Rules']
data = data[~data['committee'].isin(edit)]
print(data.groupby("committee")["committee"].count().sort_values(ascending=False))
print(len(data.groupby("committee")["committee"].count().sort_values(ascending=False)))

committee
 Codes                                                        2462
 Ways and Means                                               2036
 Education                                                    1776
 Transportation                                               1719
 Health                                                       1417
 Governmental Operations                                      1206
 Judiciary                                                    1162
 Environmental Conservation                                   1022
 Higher Education                                             1011
 Local Government                                              888
 Corporations, Authorities and Commissions                     826
 Insurance                                                     819
 Labor                                                         746
 Consumer Affairs and Protection                               677
 Governmental Employees                             

In [28]:
#split into training and testing datasets
from sklearn.model_selection import train_test_split
X = data['description']
y = data['committee']

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2, random_state = 0)

In [29]:
#initialize TFIDF in order to vectorize the words in each bill's description
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer(stop_words=stop,
                        tokenizer=tokenizer_porter,
                        preprocessor=preprocessor)
 



In [31]:
#run TFIDF vectorizer and use logistic regression as the classifier
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import re

# A pipeline is what chains several steps together, once the initial exploration is done. 
# For example, some codes are meant to transform features — normalise numericals, or turn text into vectors, 
# or fill up missing data, they are transformers; other codes are meant to predict variables by fitting an algorithm,
# they are estimators. Pipeline chains all these together which can then be applied to training data
clf = Pipeline([('vect', tfidf),
                ('clf', LogisticRegression(random_state=0))])

clf.fit(X_train, y_train)


/Users/tienmai/anaconda3/lib/python3.7/site-packages/sklearn/feature_extraction/text.py:300: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['abov', 'ani', 'becaus', 'befor', 'doe', 'dure', 'ha', 'hi', 'onc', 'onli', 'ourselv', 'themselv', 'thi', 'veri', 'wa', 'whi', 'yourselv'] not in stop_words.
  'stop_words.' % sorted(inconsistent))
/Users/tienmai/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/tienmai/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


Pipeline(memory=None,
         steps=[('vect',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=<function preprocessor at 0x1198d8598>,
                                 smooth_idf=True,
                                 stop_words=['i', 'me', 'my', 'myself', 'we'...
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function tokenizer_porter at 0x1196a6840>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
            

In [32]:
y_predict = clf.predict(X_test)

In [33]:
#check the accuracy score
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import multilabel_confusion_matrix
accuracy_score(y_test, y_predict)

0.7081236520488857

In [34]:

# classification_report
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.metrics import multilabel_confusion_matrix

accuracy_score(y_test, y_predict)
classification_report(y_test, y_predict).split('\n')

['                                                            precision    recall  f1-score   support',
 '',
 '                                               Agriculture       0.67      0.52      0.59        82',
 '                                                     Codes       0.71      0.81      0.75       488',
 '                           Consumer Affairs and Protection       0.59      0.57      0.58       123',
 '                 Corporations, Authorities and Commissions       0.68      0.65      0.66       157',
 ' Economic Development, Job Creation, Commerce and Industry       0.73      0.47      0.57        95',
 '                                                 Education       0.81      0.90      0.85       345',
 '                                              Election Law       0.87      0.82      0.84       119',
 '                                Environmental Conservation       0.71      0.76      0.74       192',
 '                                    Governmental Employee

In [35]:
print(myarrays)

[' Agriculture' ' Codes' ' Consumer Affairs and Protection'
 ' Corporations, Authorities and Commissions'
 ' Economic Development, Job Creation, Commerce and Industry' ' Education'
 ' Election Law' ' Environmental Conservation' ' Finance'
 ' Governmental Employees' ' Governmental Operations' ' Health'
 ' Higher Education' ' Housing' ' Insurance'
 ' Investigations and Government Operations' ' Judiciary' ' Labor'
 ' Local Government' ' Real Property Taxation' ' Rules' ' Transportation'
 ' Ways and Means']


In [36]:
#print out confusion matrix
cmt=pd.DataFrame(confusion_matrix(y_test, y_predict,labels=myarrays))
cmt.index=myarrays
cmt.columns=myarrays
cmt.to_csv('./Downloads/Data/confusion.csv')
print(y_test.shape)
print(len(myarrays))
print(myarrays)
cmt


(4173,)
23
[' Agriculture' ' Codes' ' Consumer Affairs and Protection'
 ' Corporations, Authorities and Commissions'
 ' Economic Development, Job Creation, Commerce and Industry' ' Education'
 ' Election Law' ' Environmental Conservation' ' Finance'
 ' Governmental Employees' ' Governmental Operations' ' Health'
 ' Higher Education' ' Housing' ' Insurance'
 ' Investigations and Government Operations' ' Judiciary' ' Labor'
 ' Local Government' ' Real Property Taxation' ' Rules' ' Transportation'
 ' Ways and Means']


,Agriculture,Codes,Consumer Affairs and Protection,"Corporations, Authorities and Commissions","Economic Development, Job Creation, Commerce and Industry",Education,Election Law,Environmental Conservation,Finance,Governmental Employees,...,Housing,Insurance,Investigations and Government Operations,Judiciary,Labor,Local Government,Real Property Taxation,Rules,Transportation,Ways and Means
Agriculture,43,6,4,0,0,0,0,8,0,0,...,0,1,0,0,0,2,1,0,3,9
Codes,4,394,8,1,3,2,0,3,0,2,...,1,3,0,16,4,0,1,0,21,5
Consumer Affairs and Protection,2,14,70,2,2,1,0,4,0,0,...,1,0,0,2,1,1,0,0,8,4
"Corporations, Authorities and Commissions",0,6,2,102,1,2,2,2,0,0,...,0,0,0,5,1,6,0,0,4,10
"Economic Development, Job Creation, Commerce and Industry",0,8,6,3,45,2,0,3,0,1,...,0,0,0,0,1,1,1,0,3,8
Education,1,2,1,1,0,309,0,2,0,1,...,0,0,0,2,0,0,1,0,3,4
Election Law,0,5,0,1,0,4,98,0,0,0,...,0,0,0,3,0,2,0,0,1,1
Environmental Conservation,2,7,6,2,1,0,0,146,0,0,...,0,1,0,4,1,2,1,0,7,6
Finance,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
Governmental Employees,0,3,0,0,0,5,0,0,0,86,...,0,3,0,4,3,5,0,0,1,7


In [37]:
#Instead of logistic regression, I tested SVC as a classifier. The accuracy score is slightly higher by aronud 2%
from sklearn.svm import LinearSVC
from sklearn.datasets import make_classification
# clf = LinearSVC(random_state=0, tol=1e-5)
clf = Pipeline([('vect', tfidf),
                ('lsvc', LinearSVC(random_state=0))])
clf.fit(X_train, y_train)
y_predict = clf.predict(X_test)
print(accuracy_score(y_test, y_predict))


0.7227414330218068
